<a href="https://colab.research.google.com/github/HeshamKhaled12/Music-Recommdation-System./blob/main/Music_Recommendation_System_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install spotipy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_id='cef9becce5514d4db83491f149ac3836'
client_secret='dcd70aec647c491e8e965d6cc61ae0d6'
client_credentials_manager=SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)
sp=spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
def get_track_name(track_name):
  tname=track_name
  results=sp.search(q=tname,type='track',limit=1)
  if results['tracks']['items']:
    track=results['tracks']['items'][0]
    track_uri=track['uri']
    track_uri2=track_uri
    track_features=sp.audio_features(track_uri2)
  else:
    print('Track not found')

  return track_features



In [ ]:
def df_new_track(track_name):
  ttname=track_name
  get_track=get_track_name(ttname)
  new_track=pd.DataFrame(get_track)
  new_track_drops=["type","id","track_href","analysis_url",'uri','time_signature']
  new_track=new_track.drop(columns=new_track_drops,axis=1)
  new_track['name']=ttname
  new_track=new_track.sort_index(axis=1)

  return new_track




In [ ]:
def reading_data (track_name):
  properties=df_new_track(track_name)
  properties.reset_index(inplace=True,drop=True)
  data = pd.read_csv ('data.csv',on_bad_lines='skip')
  data=data.drop_duplicates(subset=['name'])
  dropping=["artists","id","release_date",'popularity','explicit','year']
  data=data.drop(columns=dropping , axis=1)
  data.reset_index(inplace=True,drop=True)
  data=data.sort_index(axis=1)
  data['instrumentalness']=data['instrumentalness'].astype(float)


  named_data=pd.concat([data,properties],axis=0)



  nd=named_data.drop(['name'], axis=1)
  normalize=nd[[ 'duration_ms', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'tempo','valence']]
  names=[ 'duration_ms',  'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'tempo','valence']

  orignal_names=named_data[names].columns


  scaler=MinMaxScaler()
  normalized_data=scaler.fit_transform(nd[names])
  normalized= pd.DataFrame(normalized_data,columns=orignal_names)





  return normalized,named_data

In [ ]:
def recommndation(track_name,k):
  recommend,name_of_data=reading_data(track_name)
  name_of_data = name_of_data[~name_of_data.index.duplicated(keep='first')]

  x=recommend.iloc[-1]

  x=np.array(x)
  x=x.reshape(1, -1)

  sim=[]
  for i in tqdm(range (len(recommend))):
    y=np.array(recommend.iloc[i]).reshape(1, -1)
    cs=cosine_similarity(y, x)[0][0]
    sim.append(cs)

  sim_names=pd.DataFrame({'similar':sim,})


  final=pd.concat([sim_names,name_of_data['name']],axis=1)

  final=final.sort_values(by=['similar','name'],ascending=False)


  return print(final.iloc[1:1+k,1:1+k])





In [ ]:
recommndation('moonlight',10)

100%|██████████| 133639/133639 [01:10<00:00, 1898.70it/s]


                              name
53289                  Hey Memphis
82797                 Popular Girl
24386   Wear Your Love Like Heaven
68261              How Sweet It Is
44350                    La Vuelta
44595      Star of the County Down
65209            Ela edo konta mou
19441                  Little Pony
125462     I'll Have To Let Him Go
96906      El Baile de la Cachucha
